In [45]:
import pandas as pd
import numpy as np
import random
from pandas import json_normalize
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# Parameter for connection
server = 'DESKTOP-AL8SAM5'
database = 'EmartShopping'
username = 'sa'
password = 'huan181102'

# Create SQLAlchemy engine
engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server')

# Read data and convert to Dataframe
df_products = pd.read_sql_query('SELECT p_name, p_price FROM Products', engine)

def generate_random_products(df_products):

    random_products = []

    # Create random quantity sample product
    n_sample = np.random.randint(1,11)

    # Random sample products
    df_products = df_products.sample(n=n_sample, replace=False)

    # Create list random products
    for i in range(df_products.shape[0]):

        random_product = {}

        random_product['p_name'] = df_products['p_name'].iloc[i]
        random_product['p_price'] = df_products['p_price'].iloc[i]
        random_product['p_quantity'] = np.random.randint(1,6)

        random_products.append(random_product)

    return random_products

def generate_random_time():

    # Get the current date
    now = datetime.now()
    start = datetime(now.year, now.month, now.day, 7)

    # Parameters for random time
    random_hours = np.random.randint(0, 16)
    random_minutes = np.random.randint(0, 61)
    random_seconds = np.random.randint(0, 61)
    
    # Create random time with start and parameter
    random_time = start + timedelta(hours=random_hours) + timedelta(minutes=random_minutes) + timedelta(seconds=random_seconds)
    
    # Format random time
    random_time = random_time.strftime('%Y-%m-%d %H:%M:%S')

    return random_time

def generate_orders(total):

    orders = []

    for o_id in range(1, total+1):

        o_date = generate_random_time()

        random_products = generate_random_products(df_products)

        for order in random_products:

            order['o_id'] = o_date.split(' ')[0].replace('-', '') + str(o_id) 
            order['o_date'] = o_date

            orders.append(order)
    
    df_orders = json_normalize(orders)
        
    return df_orders

df_orders = generate_orders(100)

df_orders.head(10)

,p_name,p_price,p_quantity,o_id,o_date
0,Khăn Mặt Cotton Y622 122G 34*80Cm - Màu Hồng,39900,1,202405301,2024-05-30 11:00:54
1,Cà Phê Trung Nguyên Rang Xay Khát Vọng I 500G,86900,3,202405301,2024-05-30 11:00:54
2,Hành Bột Vipep Hũ 70G,20800,2,202405301,2024-05-30 11:00:54
3,Khăn Ướt Siêu Dày Có Nắp No Brand 100 Miếng,35000,4,202405301,2024-05-30 11:00:54
4,Túi Đựng Thực Phẩm Emart 20*30Cm 350G,31900,4,202405301,2024-05-30 11:00:54
5,Bình Đun Siêu Tốc Sunhouse SHD1182 1.8L,189000,4,202405301,2024-05-30 11:00:54
6,Cháo Asuzac Gà Ly 36G,14500,1,202405302,2024-05-30 11:25:47
7,Nước Giặt Comfort Dưỡng Vải Đa Năng Thiên Nhiê...,140400,5,202405302,2024-05-30 11:25:47
8,Bánh Gạo An Vị Tự Nhiên 151.2G,15400,1,202405302,2024-05-30 11:25:47
9,Bột Nghệ Natas Hũ 40G,17600,1,202405302,2024-05-30 11:25:47


In [4]:
df_orders_drop = df_orders[['o_id', 'p_id']]

In [5]:
df_orders_drop

,o_id,p_id
0,1,Nước Mắm Thanh Hà 40N 250Ml
1,1,Bún Tôm Vifon Gói 65G
2,1,Nước Giặt Lix Matic Hương Nước Hoa Túi 3.5Kg
3,2,Trà Sữa Kirin Tea Break Chai 345Ml
4,2,Bơ Thực Vật Margarine Tường An 80G
...,...,...
499620,200005,Lô 2 Bàn Chải Đánh Răng Oral Clean Spiral Carb...
499621,200005,Dầu Xả Dược Liệu Nguyên Xuân 150Ml
499622,200005,Bánh Xếp Mandu Bibigo Healthy Tôm Gói 300G
499623,200006,Tương Ớt Nam Dương Hàng Việt Chai Nhựa 800G


In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori

grouped = df_orders_drop.groupby('o_id')['p_id'].apply(list)
print(grouped)
list_grouped = list(grouped)
print(list_grouped)
te = TransactionEncoder()
te_ary = te.fit(list_grouped).transform(list_grouped)
df_new = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(df_new, min_support=0.1, use_colnames=True)
print(frequent_itemsets)

df_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

df_rules.head()